In [1]:
# Empheris of where things were
# 
# 18-04-2023    IGH

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy import timeseries as ts
from sunpy.time import parse_time
from sunpy.coordinates import frames, get_horizons_coord
import sunpy.map
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
import datetime
import pandas as pd
import os
from pathlib import Path
import glob
from stixdcpy.quicklook import LightCurves
from stixdcpy import auxiliary as aux

import nustar_pysolar as nustar
from scipy import ndimage

import matplotlib.colors as colors
from matplotlib import patches
from matplotlib.colors import LogNorm
from sunpy.coordinates import Helioprojective

import nsx_func

import warnings
warnings.simplefilter('ignore')

plt.rcParams.update({'font.size': 20,'font.family':"sans-serif",\
            'font.sans-serif':"Arial",'mathtext.default':"regular"})

In [3]:
# List of all the NuSTAR observations, with my naming, 
# based off first day of observing per campaign
dobs=['20140910','20141101','20141211',
    '20150429','20150901',
    '20160219','20160422','20160726',
    '20170321','20170821','20170911','20171010',
    '20180529','20180907','20180928',
    '20190112','20190412','20190425','20190702',
    '20200129','20200221','20200606','20200912',
    '20210108','20210429','20210720','20210730','20211117',
    '20220224','20220603','20220906','20221209',
    '20230318']

# Data directory
# ddir=str(Path.home())+'/data/heasarc_nustar/'
ddir='/Volumes/Samsung_T5/data/heasarc_nustar/'

In [4]:
# Plot emph
for icamp in [21,23,24,25,26,27,28,29,30,31,32]:
# for icamp in [21]:
    maindir=ddir+'ns_'+dobs[icamp]+'/'
    print(icamp, ' --- ',maindir)
    # Most start with 20 or 80 or 90?
    ids = [f.name for f in os.scandir(maindir) \
        if (f.is_dir() and (f.name.startswith('20') or f.name.startswith('80') or f.name.startswith('90')))]
    ids=sorted(ids)
    # Filter out random other dirs
    ids=[f for f in ids if not f.endswith('_new')]
    # Remove the mosiac from 20220603
    if (dobs[icamp] == '20220603'):
        ids=[f for f in ids if f.startswith('2062400')]      
    print("#: ",len(ids),' --- ',ids)

    # Do the map for each pointing
    for nsid in ids:
        print(nsid)

        df_test=nsx_func.nsrate(maindir=maindir,nsid=nsid,lvt=True,englow=2,enghigh=10)
        df10 = df_test.resample('10s', level=0).mean()
        # Time range to nearest 5mins
        mint=df10.index[0].to_pydatetime()
        mint-=datetime.timedelta(minutes=5,seconds=mint.second)
        mint-=datetime.timedelta(minutes=mint.minute % 5)
        maxt=df10.index[-1].to_pydatetime()
        maxt+=datetime.timedelta(minutes=5,seconds=60-maxt.second)
        maxt-=datetime.timedelta(minutes=maxt.minute % 5)
        tr=[mint,maxt]

        eph=aux.Ephemeris.from_sdc(start_utc=tr[0],end_utc=tr[1],steps=10)

        fig, ax= plt.subplots(1,figsize=(5, 5))
        plt.rcParams.update({'font.size': 18})
        # emph.peek()
        sun = patches.Circle((0.0, 0.0), 0.25, alpha=0.8, fc='darkorange')
        plt.text(0, 0, 'Sun',fontweight='bold',fontsize=20)
        ax.add_patch(sun)
        earth = patches.Circle((-1, 0), 0.12, alpha=0.8, fc='lightseagreen')
        ax.add_patch(earth)
        ax.text(-1, 0, 'Earth',fontweight='bold',fontsize=20)
        ax.plot(eph.data['orbit']['x'][-1], eph.data['orbit']['y'][-1], '+',ms=21,mew=2,
            color='steelblue',label=f'{mint:%d-%b-%Y %H:%M:%S}')
        ax.text(eph.data['orbit']['x'][-1], eph.data['orbit']['y'][-1], 'SO',
            color='steelblue',fontweight='bold',fontsize=20)
        ax.set_xlabel('x [au]')
        ax.set_ylabel('y [au]')
        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.2, 1.2)
        ax.set_aspect('equal')
        ax.grid()
        plt.legend(loc=2)
        fig.subplots_adjust(bottom=0.14, top=0.92, left=0.2, right=0.98)
        plt.savefig('figs/ns_'+dobs[icamp]+f'/eph_{mint:%Y%m%d_%H%M}_'+nsid+'_ngs.png')
        plt.close()
        # break
    # break


21  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20200606/
#:  10  ---  ['20611001001', '20611002001', '20611003001', '20611004001', '20611005001', '20611006001', '20611007001', '20611008001', '20611009001', '20611010001']
20611001001
20611002001
20611003001
20611004001
20611005001
20611006001
20611007001
20611008001
20611009001
20611010001
23  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20210108/
#:  12  ---  ['20612001001', '20612002001', '20612003001', '20612004001', '20613001001', '20613002001', '20613003001', '20613004001', '20613005001', '20614001001', '20614002001', '20614003001']
20612001001
20612002001
20612003001
20612004001
20613001001
20613002001
20613003001
20613004001
20613005001
20614001001
20614002001
20614003001
24  ---  /Volumes/Samsung_T5/data/heasarc_nustar/ns_20210429/
#:  16  ---  ['20615001001', '20615002001', '20615003001', '20615004001', '20615005001', '20616001001', '20616002001', '20616003001', '20616004001', '20616005001', '20617001001', '20617002001'